# **Text**

##### English

In [1]:
!pip uninstall torch -y
!pip install torch --upgrade
!pip install transformers

Found existing installation: torch 2.6.0
Uninstalling torch-2.6.0:
  Successfully uninstalled torch-2.6.0
  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl (766.7 MB)


In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


2.6.0+cu124
True


In [ ]:
from transformers import pipeline

# مدل عمومی و در دسترس
model_name = "SamLowe/roberta-base-go_emotions"
classifier = pipeline("text-classification", model=model_name, tokenizer=model_name)

# تست جمله
sentence = "I'm so tired and hopeless today."
results = classifier(sentence)
print(results)


config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'disappointment', 'score': 0.5366367101669312}]


##### Farsi

In [ ]:
from transformers import pipeline

# مدل عمومی فارسی برای احساسات
model_name = "HooshvareLab/bert-fa-base-uncased-sentiment-snappfood"
classifier = pipeline("text-classification", model=model_name, tokenizer=model_name)

# تست جمله فارسی
sentence = "از زندگی خیلی خسته‌ام و حس خوبی ندارم."
results = classifier(sentence)
print(results)


config.json:   0%|          | 0.00/621 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/651M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/651M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'SAD', 'score': 0.9199081063270569}]


In [ ]:
from transformers import pipeline

# مدل عمومی فارسی برای احساسات
model_name = "HooshvareLab/bert-fa-base-uncased-sentiment-snappfood"
classifier = pipeline("text-classification", model=model_name, tokenizer=model_name)

# تست جمله فارسی
sentence = "اونقدر قلبم قبره که راز تو توش دفن."
results = classifier(sentence)
print(results)


Device set to use cuda:0


[{'label': 'SAD', 'score': 0.6089940071105957}]


In [ ]:
from transformers import pipeline

model_name = "HooshvareLab/bert-fa-base-uncased-sentiment-deepsentipers-multi"
classifier = pipeline("text-classification", model=model_name, tokenizer=model_name)

sentence = "از زندگی خیلی خسته‌ام و حس خوبی ندارم."
results = classifier(sentence)
print(results)


Device set to use cuda:0


[{'label': 'furious', 'score': 0.9774502515792847}]


# Voice

In [3]:
!pip install pydub
!apt-get install ffmpeg -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [4]:
!pip install torchaudio librosa transformers datasets

In [ ]:
from transformers import pipeline
from pydub import AudioSegment

# لود مدل تحلیل احساس از صوت
model_id = "superb/wav2vec2-base-superb-er"
emotion_pipeline = pipeline("audio-classification", model=model_id)

# تبدیل mp3 به wav با تنظیمات درست
audio = AudioSegment.from_file("/content/input.mp3")
audio = audio.set_frame_rate(16000).set_channels(1)
audio.export("/content/sample.wav", format="wav")

# تست مدل روی فایل صوتی wav
audio_path = "/content/sample.wav"
results = emotion_pipeline(audio_path)
print(results)

Device set to use cuda:0


[{'score': 0.8297052383422852, 'label': 'ang'}, {'score': 0.15069030225276947, 'label': 'sad'}, {'score': 0.01826886087656021, 'label': 'neu'}, {'score': 0.0013356173876672983, 'label': 'hap'}]


In [5]:
!pip install gradio

In [ ]:
from transformers import pipeline
from pydub import AudioSegment
import gradio as gr

# مدل صوتی (با ۴ احساس اصلی)
audio_model = pipeline("audio-classification", model="superb/wav2vec2-base-superb-er")

# مدل متنی انگلیسی (با احساسات بیشتر مثل joy, love, anger, optimism, fear, etc.)
text_model = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

# آماده‌سازی صوت برای مدل صوتی
def convert_audio(file):
    audio = AudioSegment.from_file(file)
    audio = audio.set_frame_rate(16000).set_channels(1)
    out_path = "/content/temp.wav"
    audio.export(out_path, format="wav")
    return out_path

# تحلیل احساس صوتی + متنی
def analyze(text, audio_file):
    results = {}

    # متن
    if text:
        text_res = text_model(text)
        text_emotions = sorted(text_res[0], key=lambda x: -x["score"])[:3]
        results["📄 احساسات متن"] = text_emotions

    # صدا
    if audio_file:
        audio_path = convert_audio(audio_file)
        audio_res = audio_model(audio_path)
        results["🔊 احساسات صدا"] = audio_res

    return results

# رابط Gradio
gr.Interface(
    fn=analyze,
    inputs=[
        gr.Textbox(label="📝 متن (انگلیسی یا ترجمه‌شده)"),
        gr.Audio(label="🎤 ویس یا آهنگ (MP3/WAV)", type="filepath")
    ],
    outputs="json",
    title="سیستم ترکیبی تشخیص احساسات (صوت + متن)",
    description="این ابزار احساسات را از صدا و متن به‌صورت ترکیبی تشخیص می‌دهد. مناسب برای سیستم‌های پیشنهاد موسیقی."
).launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.6 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Device set to use cuda:0


config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use cuda:0


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://14c33cca7dbf28d0f3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# picture

In [6]:
!pip install opencv-python keras numpy tensorflow

In [7]:
!pip install --upgrade tensorflow

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # اجرای روی CPU

import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# دانلود مدل
!wget https://github.com/oarriaga/face_classification/raw/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5 -O emotion_model.h5

# لود مدل بدون کامپایل
model = load_model('emotion_model.h5', compile=False)

emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

img_path = "/content/face4.jpg"
frame = cv2.imread(img_path)
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
faces = face_classifier.detectMultiScale(gray, 1.3, 5)

for (x, y, w, h) in faces:
    roi_gray = gray[y:y+h, x:x+w]
    roi_gray = cv2.resize(roi_gray, (64, 64), interpolation=cv2.INTER_AREA)

    if np.sum([roi_gray]) != 0:
        roi = roi_gray.astype('float32') / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        roi = roi.reshape((1, 64, 64, 1))

        preds = model.predict(roi)[0]
        label = emotion_labels[np.argmax(preds)]
        confidence = np.max(preds)
        print(f"🧠 Detected Emotion: {label} ({confidence:.2f})")
    else:
        print("🚫 No face detected.")


--2025-04-19 22:10:43--  https://github.com/oarriaga/face_classification/raw/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/oarriaga/face_classification/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5 [following]
--2025-04-19 22:10:43--  https://raw.githubusercontent.com/oarriaga/face_classification/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 872856 (852K) [application/octet-stream]
Saving to: ‘emotion_model.h5’

emoti

In [2]:
from transformers import pipeline
from pydub import AudioSegment
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import gradio as gr
import os

# اجرای مدل فقط روی CPU (امن‌تر)
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# مدل‌ها

# text:
text_en = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)
text_fa = pipeline("text-classification", model="HooshvareLab/bert-fa-base-uncased-sentiment-snappfood")
# audio:
audio_model = pipeline("audio-classification", model="superb/wav2vec2-base-superb-er")
# picture:
!wget https://github.com/oarriaga/face_classification/raw/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5 -O emotion_model.h5
face_model = load_model("emotion_model.h5", compile=False)

# لیبل‌های چهره
face_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

def detect_language(text):
    return 'fa' if any('\u0600' <= ch <= '\u06FF' for ch in text) else 'en'

def analyze_text(text):
    lang = detect_language(text)
    if lang == 'fa':
        return text_fa(text)
    else:
        results = text_en(text)
        return sorted(results[0], key=lambda x: -x["score"])[:3]

def analyze_audio(audio_file):
    audio = AudioSegment.from_file(audio_file)
    audio = audio.set_frame_rate(16000).set_channels(1)
    audio.export("temp.wav", format="wav")
    return audio_model("temp.wav")

def analyze_image(image):
    img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (64, 64), interpolation=cv2.INTER_AREA)
        if np.sum([roi_gray]) == 0:
            continue
        roi = roi_gray.astype("float32") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0).reshape((1, 64, 64, 1))
        preds = face_model.predict(roi)[0]
        top = np.argmax(preds)
        return [{"label": face_labels[top], "score": float(preds[top])}]
    return [{"label": "No face found", "score": 0}]

def full_analysis(text, audio, image):
    results = {}
    if text:
        results["📄 متن"] = analyze_text(text)
    if audio:
        results["🔊 ویس"] = analyze_audio(audio)
    if image is not None:
        results["🖼️ چهره"] = analyze_image(image)
    return results

# رابط کاربری Gradio
gr.Interface(
    fn=full_analysis,
    inputs=[
        gr.Textbox(label="📝 متن (فارسی یا انگلیسی)"),
        gr.Audio(label="🎤 ویس یا آهنگ", type="filepath"),
        gr.Image(label="📷 تصویر چهره")
    ],
    outputs="json",
    title="🎵 دستیار هوشمند احساسات و موسیقی",
    description="تحلیل احساسات چندرسانه‌ای از متن، صوت و تصویر با ترکیب خروجی‌ها برای پیشنهاد موسیقی"
).launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu
Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Device set to use cpu


--2025-04-19 22:11:31--  https://github.com/oarriaga/face_classification/raw/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/oarriaga/face_classification/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5 [following]
--2025-04-19 22:11:31--  https://raw.githubusercontent.com/oarriaga/face_classification/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 872856 (852K) [application/octet-stream]
Saving to: ‘emotion_model.h5’

emoti

# Emotion-Based Recommendation

In [3]:
# prepare_emotion_dataset.py

import os
import json
import random

# شبیه‌سازی داده‌های تمرینی از متن احساسی و دنباله موسیقی
# در پروژه واقعی، این باید از یک دیتاست واقعی مثل MAESTRO استخراج شود.

emotions = [
    "happy", "sad", "angry", "fear", "surprise", "calm", "love", "lonely"
]

notes = ["C4", "D4", "E4", "F4", "G4", "A4", "B4", "C5", "D5"]
durations = [0.25, 0.5, 1.0, 1.5]

# تولید 200 نمونه تمرینی برای شروع ساده
samples = []
for _ in range(200):
    emotion = random.choice(emotions)
    prompt = f"Emotion: {emotion}"
    sequence = []
    for _ in range(random.randint(8, 20)):
        pitch = random.choice(notes)
        dur = random.choice(durations)
        sequence.append(f"NOTE_{pitch}")
        sequence.append(f"DUR_{dur}")

    samples.append({
        "text": prompt,
        "tokens": sequence
    })

os.makedirs("dataset", exist_ok=True)
with open("dataset/tokenized_dataset.json", "w") as f:
    json.dump(samples, f, indent=2)

print("✅  دیتاست آموزشی با موفقیت ساخته شد → dataset/tokenized_dataset.json")


✅  دیتاست آموزشی با موفقیت ساخته شد → dataset/tokenized_dataset.json


In [7]:
# train_emotion_music_model.py

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import json
from transformers import BertTokenizer
from tqdm import tqdm
import os
from collections import Counter

# ✅ دیتاست
TOKENIZER_MODEL = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(TOKENIZER_MODEL)

class EmotionMusicDataset(Dataset):
    def __init__(self, json_path, max_text_len=64, max_music_len=128):
        with open(json_path) as f:
            self.data = json.load(f)
        self.max_text_len = max_text_len
        self.max_music_len = max_music_len

        all_tokens = set()
        for item in self.data:
            all_tokens.update(item['tokens'])

        self.vocab = {t: i + 2 for i, t in enumerate(sorted(all_tokens))}  # +2 for PAD=0, START=1
        self.vocab['<PAD>'] = 0
        self.vocab['<START>'] = 1
        self.inv_vocab = {v: k for k, v in self.vocab.items()}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']
        tokens = item['tokens'][:self.max_music_len - 1]

        text_ids = tokenizer(text, padding='max_length', truncation=True, max_length=self.max_text_len, return_tensors='pt')['input_ids'].squeeze(0)

        input_tokens = ['<START>'] + tokens
        target_tokens = tokens + ['<PAD>']

        input_ids = torch.tensor([self.vocab[t] for t in input_tokens] + [0] * (self.max_music_len - len(input_tokens)))
        target_ids = torch.tensor([self.vocab[t] for t in target_tokens] + [0] * (self.max_music_len - len(target_tokens)))

        return text_ids, input_ids, target_ids

# ✅ مدل Transformer پیشرفته‌تر
class TransformerMusicModel(nn.Module):
    def __init__(self, vocab_size, text_vocab_size, embed_dim=256, num_heads=4, num_layers=4, hidden_dim=512):
        super().__init__()
        self.text_embedding = nn.Embedding(text_vocab_size, embed_dim)
        self.music_embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_encoder = nn.Parameter(torch.randn(1, 256, embed_dim))
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(embed_dim, vocab_size)

    def forward(self, text_input, music_input):
        text_emb = self.text_embedding(text_input)
        music_emb = self.music_embedding(music_input)
        x = torch.cat((text_emb, music_emb), dim=1)
        x = x + self.pos_encoder[:, :x.size(1), :]
        x = self.transformer(x)
        return self.fc_out(x)

# ✅ آموزش مدل + تحلیل خروجی برای ارزیابی تنوع و ریتم
if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    dataset = EmotionMusicDataset("dataset/tokenized_dataset.json")
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
    model = TransformerMusicModel(len(dataset.vocab), text_vocab_size=30522).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
    loss_fn = nn.CrossEntropyLoss(ignore_index=0)

    for epoch in range(10):
        model.train()
        total_loss = 0
        token_stats = Counter()

        for text_ids, input_ids, target_ids in tqdm(dataloader):
            text_ids = text_ids.to(device)
            input_ids = input_ids.to(device)
            target_ids = target_ids.to(device)

            optimizer.zero_grad()
            out = model(text_ids, input_ids)
            loss = loss_fn(out[:, text_ids.size(1):].reshape(-1, out.shape[-1]), target_ids.reshape(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            # تجزیه و تحلیل تنوع توکن‌ها (فقط یک batch برای مثال)
            predicted_ids = out.argmax(dim=-1).cpu().numpy()
            for row in predicted_ids:
                tokens = [dataset.inv_vocab.get(idx, '<UNK>') for idx in row[text_ids.size(1):]]
                for tok in tokens:
                    if tok.startswith("NOTE") or tok.startswith("DUR"):
                        token_stats[tok] += 1

        print(f"🎯 Epoch {epoch+1}: Loss = {total_loss:.4f}")
        print(f"🎼 NOTE Tokens: {[t for t in token_stats.items() if 'NOTE' in t[0]]}")
        print(f"⏱️ DUR Tokens: {[t for t in token_stats.items() if 'DUR' in t[0]]}")

    torch.save(model.state_dict(), "emotion_music_model.pth")
    with open("emotion_vocab.json", "w") as f:
        json.dump(dataset.vocab, f)
    print("✅ مدل ذخیره شد → emotion_music_model.pth")


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
100%|██████████| 25/25 [00:13<00:00,  1.88it/s]


🎯 Epoch 1: Loss = 49.5256
🎼 NOTE Tokens: [('NOTE_C4', 554), ('NOTE_D4', 1240), ('NOTE_A4', 1607), ('NOTE_E4', 289), ('NOTE_G4', 462), ('NOTE_F4', 1392), ('NOTE_B4', 406), ('NOTE_C5', 231), ('NOTE_D5', 603)]
⏱️ DUR Tokens: [('DUR_1.0', 2428), ('DUR_0.25', 11409), ('DUR_1.5', 1928), ('DUR_0.5', 2964)]


100%|██████████| 25/25 [00:09<00:00,  2.57it/s]


🎯 Epoch 2: Loss = 45.6284
🎼 NOTE Tokens: [('NOTE_C4', 81), ('NOTE_G4', 1682), ('NOTE_D4', 919), ('NOTE_D5', 937), ('NOTE_A4', 1570), ('NOTE_E4', 378), ('NOTE_F4', 1121), ('NOTE_B4', 352), ('NOTE_C5', 30)]
⏱️ DUR Tokens: [('DUR_1.5', 2506), ('DUR_1.0', 5679), ('DUR_0.25', 7516), ('DUR_0.5', 2829)]


100%|██████████| 25/25 [00:09<00:00,  2.54it/s]


...

🎯 Epoch 8: Loss = 44.8357
🎼 NOTE Tokens: [('NOTE_D4', 689), ('NOTE_G4', 2062), ('NOTE_A4', 659), ('NOTE_F4', 1270), ('NOTE_C5', 336), ('NOTE_E4', 1892), ('NOTE_D5', 1727), ('NOTE_C4', 1819), ('NOTE_B4', 916)]
⏱️ DUR Tokens: [('DUR_1.0', 2014), ('DUR_0.5', 2146), ('DUR_0.25', 9287), ('DUR_1.5', 783)]


100%|██████████| 25/25 [00:09<00:00,  2.68it/s]


🎯 Epoch 9: Loss = 44.9524
🎼 NOTE Tokens: [('NOTE_E4', 1746), ('NOTE_B4', 630), ('NOTE_C5', 334), ('NOTE_D4', 921), ('NOTE_A4', 1286), ('NOTE_D5', 2236), ('NOTE_G4', 2077), ('NOTE_C4', 893), ('NOTE_F4', 1830)]
⏱️ DUR Tokens: [('DUR_0.5', 1596), ('DUR_1.5', 1027), ('DUR_0.25', 9281), ('DUR_1.0', 1743)]


100%|██████████| 25/25 [00:09<00:00,  2.53it/s]

🎯 Epoch 10: Loss = 44.5848
🎼 NOTE Tokens: [('NOTE_C4', 1342), ('NOTE_E4', 649), ('NOTE_D5', 2666), ('NOTE_G4', 2043), ('NOTE_F4', 2855), ('NOTE_B4', 1629), ('NOTE_C5', 344), ('NOTE_D4', 184), ('NOTE_A4', 372)]
⏱️ DUR Tokens: [('DUR_0.25', 8417), ('DUR_0.5', 1700), ('DUR_1.0', 1458), ('DUR_1.5', 1941)]
✅ مدل ذخیره شد → emotion_music_model.pth


In [8]:
# train_emotion_music_model.py

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import json
from transformers import BertTokenizer
from tqdm import tqdm
import os
from collections import Counter

# ✅ دیتاست
TOKENIZER_MODEL = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(TOKENIZER_MODEL)

class EmotionMusicDataset(Dataset):
    def __init__(self, json_path, max_text_len=64, max_music_len=128):
        with open(json_path) as f:
            self.data = json.load(f)
        self.max_text_len = max_text_len
        self.max_music_len = max_music_len

        all_tokens = set()
        for item in self.data:
            all_tokens.update(item['tokens'])

        self.vocab = {t: i + 2 for i, t in enumerate(sorted(all_tokens))}  # +2 for PAD=0, START=1
        self.vocab['<PAD>'] = 0
        self.vocab['<START>'] = 1
        self.inv_vocab = {v: k for k, v in self.vocab.items()}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']
        tokens = item['tokens'][:self.max_music_len - 1]

        text_ids = tokenizer(text, padding='max_length', truncation=True, max_length=self.max_text_len, return_tensors='pt')['input_ids'].squeeze(0)

        input_tokens = ['<START>'] + tokens
        target_tokens = tokens + ['<PAD>']

        input_ids = torch.tensor([self.vocab[t] for t in input_tokens] + [0] * (self.max_music_len - len(input_tokens)))
        target_ids = torch.tensor([self.vocab[t] for t in target_tokens] + [0] * (self.max_music_len - len(target_tokens)))

        return text_ids, input_ids, target_ids

# ✅ مدل Transformer پیشرفته‌تر
class TransformerMusicModel(nn.Module):
    def __init__(self, vocab_size, text_vocab_size, embed_dim=256, num_heads=4, num_layers=4, hidden_dim=512):
        super().__init__()
        self.text_embedding = nn.Embedding(text_vocab_size, embed_dim)
        self.music_embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_encoder = nn.Parameter(torch.randn(1, 256, embed_dim))
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(embed_dim, vocab_size)

    def forward(self, text_input, music_input):
        text_emb = self.text_embedding(text_input)
        music_emb = self.music_embedding(music_input)
        x = torch.cat((text_emb, music_emb), dim=1)
        x = x + self.pos_encoder[:, :x.size(1), :]
        x = self.transformer(x)
        return self.fc_out(x)

# ✅ آموزش مدل + تحلیل خروجی برای ارزیابی تنوع و ریتم
if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    dataset = EmotionMusicDataset("dataset/tokenized_dataset.json")
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
    model = TransformerMusicModel(len(dataset.vocab), text_vocab_size=30522).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
    loss_fn = nn.CrossEntropyLoss(ignore_index=0)

    for epoch in range(10):
        model.train()
        total_loss = 0
        token_stats = Counter()

        for text_ids, input_ids, target_ids in tqdm(dataloader):
            text_ids = text_ids.to(device)
            input_ids = input_ids.to(device)
            target_ids = target_ids.to(device)

            optimizer.zero_grad()
            out = model(text_ids, input_ids)
            loss = loss_fn(out[:, text_ids.size(1):].reshape(-1, out.shape[-1]), target_ids.reshape(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            # تجزیه و تحلیل تنوع توکن‌ها (فقط یک batch برای مثال)
            predicted_ids = out.argmax(dim=-1).cpu().numpy()
            for row in predicted_ids:
                tokens = [dataset.inv_vocab.get(idx, '<UNK>') for idx in row[text_ids.size(1):]]
                for tok in tokens:
                    if tok.startswith("NOTE") or tok.startswith("DUR"):
                        token_stats[tok] += 1

        print(f"🎯 Epoch {epoch+1}: Loss = {total_loss:.4f}")
        print(f"🎼 NOTE Tokens: {[t for t in token_stats.items() if 'NOTE' in t[0]]}")
        print(f"⏱️ DUR Tokens: {[t for t in token_stats.items() if 'DUR' in t[0]]}")

    torch.save(model.state_dict(), "emotion_music_model.pth")
    with open("emotion_vocab.json", "w") as f:
        json.dump(dataset.vocab, f)
    print("✅ مدل ذخیره شد → emotion_music_model.pth")


100%|██████████| 25/25 [00:16<00:00,  1.56it/s]


🎯 Epoch 1: Loss = 51.4211
🎼 NOTE Tokens: [('NOTE_A4', 599), ('NOTE_G4', 888), ('NOTE_C5', 285), ('NOTE_C4', 150), ('NOTE_F4', 226), ('NOTE_B4', 119), ('NOTE_E4', 228), ('NOTE_D5', 367), ('NOTE_D4', 172)]
⏱️ DUR Tokens: [('DUR_1.0', 6877), ('DUR_1.5', 2619), ('DUR_0.25', 9159), ('DUR_0.5', 3813)]


100%|██████████| 25/25 [00:12<00:00,  1.94it/s]


🎯 Epoch 2: Loss = 45.5808
🎼 NOTE Tokens: [('NOTE_D4', 550), ('NOTE_F4', 1118), ('NOTE_A4', 1236), ('NOTE_C4', 751), ('NOTE_C5', 1272), ('NOTE_E4', 790), ('NOTE_G4', 1808), ('NOTE_D5', 907), ('NOTE_B4', 1918)]
⏱️ DUR Tokens: [('DUR_0.25', 7467), ('DUR_1.5', 2471), ('DUR_0.5', 1933), ('DUR_1.0', 3379)]


100%|██████████| 25/25 [00:12<00:00,  1.94it/s]


...

🎯 Epoch 8: Loss = 8.7428
🎼 NOTE Tokens: [('NOTE_D4', 451), ('NOTE_C5', 365), ('NOTE_E4', 2323), ('NOTE_F4', 5845), ('NOTE_G4', 1042), ('NOTE_C4', 3186), ('NOTE_D5', 1337), ('NOTE_A4', 2788), ('NOTE_B4', 565)]
⏱️ DUR Tokens: [('DUR_1.5', 4683), ('DUR_1.0', 1282), ('DUR_0.5', 944), ('DUR_0.25', 789)]


100%|██████████| 25/25 [00:12<00:00,  1.99it/s]


🎯 Epoch 9: Loss = 4.2306
🎼 NOTE Tokens: [('NOTE_A4', 1971), ('NOTE_B4', 689), ('NOTE_E4', 2128), ('NOTE_C5', 526), ('NOTE_C4', 4728), ('NOTE_D5', 1543), ('NOTE_F4', 3252), ('NOTE_D4', 414), ('NOTE_G4', 1559)]
⏱️ DUR Tokens: [('DUR_1.0', 1448), ('DUR_0.5', 958), ('DUR_1.5', 5586), ('DUR_0.25', 798)]


100%|██████████| 25/25 [00:14<00:00,  1.68it/s]

🎯 Epoch 10: Loss = 3.2300
🎼 NOTE Tokens: [('NOTE_D4', 452), ('NOTE_G4', 1809), ('NOTE_D5', 1414), ('NOTE_F4', 2333), ('NOTE_A4', 2104), ('NOTE_C5', 836), ('NOTE_C4', 5357), ('NOTE_E4', 3168), ('NOTE_B4', 1061)]
⏱️ DUR Tokens: [('DUR_0.5', 822), ('DUR_1.0', 1133), ('DUR_1.5', 4388), ('DUR_0.25', 723)]
✅ مدل ذخیره شد → emotion_music_model.pth


In [ ]:
# generate_from_emotion_json.py

import torch
import json
import os
from train_emotion_music_model import TransformerMusicModel
from transformers import BertTokenizer
from emotion_to_midi import tokens_to_midi  # باید بسازیمش
from midi2audio import FluidSynth

# 🔧 تنظیمات
INPUT_JSON = "input/emotion_analysis.json"
MODEL_PATH = "emotion_music_model.pth"
VOCAB_PATH = "emotion_vocab.json"
OUTPUT_MIDI = "output/generated_music.mid"
OUTPUT_MP3 = "output/generated_music.mp3"
TOKENIZER_MODEL = "bert-base-uncased"

# 🔽 احساس → متن ورودی برای مدل
def emotion_to_prompt(emotion):
    return f"Emotion: {emotion}"

# ⬇️ آماده‌سازی
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained(TOKENIZER_MODEL)

# 🔄 بارگذاری واژگان
with open(VOCAB_PATH) as f:
    vocab = json.load(f)
inv_vocab = {v: k for k, v in vocab.items()}

# 🧠 بارگذاری مدل
model = TransformerMusicModel(vocab_size=len(vocab), text_vocab_size=30522)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

# 📤 دریافت احساس
with open(INPUT_JSON) as f:
    input_data = json.load(f)
dominant_emotion = input_data.get("dominant_emotion", "calm")

# 📝 ساخت ورودی مدل
prompt = emotion_to_prompt(dominant_emotion)
text_ids = tokenizer(prompt, padding='max_length', truncation=True, max_length=64, return_tensors='pt')['input_ids'].to(device)

# ⏱️ تولید توکن موسیقی
generated = ['<START>']
max_gen_len = 128

for _ in range(max_gen_len):
    input_tokens = torch.tensor([vocab.get(tok, 0) for tok in generated], dtype=torch.long).unsqueeze(0).to(device)
    out = model(text_ids, input_tokens)
    next_token_id = out[0, -1].argmax().item()
    next_token = inv_vocab.get(next_token_id, '<PAD>')
    if next_token == '<PAD>':
        break
    generated.append(next_token)

# 🎼 تبدیل توکن‌ها به MIDI
note_tokens = [tok for tok in generated if tok.startswith("NOTE") or tok.startswith("DUR") or tok.startswith("INSTR")]
tokens_to_midi(note_tokens, OUTPUT_MIDI)

# 🎵 تبدیل MIDI به MP3
fs = FluidSynth()
fs.midi_to_audio(OUTPUT_MIDI, OUTPUT_MP3)

print(f"✅ موسیقی تولید شد و ذخیره شد در → {OUTPUT_MP3}")


# Music Generation

In [ ]:
!apt-get update -y
!apt-get install -y libasound2-dev


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [73.0 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,243 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted a

In [ ]:
!pip install magenta --no-deps

  Using cached magenta-2.1.4-py3-none-any.whl.metadata (2.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.8 MB/s eta 0:00:00


In [ ]:
!pip install note-seq

  Using cached note_seq-0.0.5-py3-none-any.whl.metadata (1.1 kB)
  Preparing metadata (setup.py) ... done
  Using cached pretty_midi-0.2.10.tar.gz (5.6 MB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.2 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26098 sha256=91b33867266bef7222526a95040146ca51d9f1f97ba791131a20dfff53071a06
  Stored in directory: /root/.cache/pip/wheels/31/d7/d9/eec6891f78cac19a693bd40ecb8365d2f4613318c145ec9816
  Created wheel for pretty-midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592286 sha256=8244f7126ad03108495aafafbfdf4df30dba0bd9dcb907a04f68666344ce3cfa
  Stored in directory: /root/.cache/pip/wheels/e6/95/ac/15ceaeb2823b04d8e638fd1495357adb8d26c00ccac9d7782e
Successfully built inter

In [ ]:
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.music.sequences_lib import concatenate_sequences
from magenta.music import sequence_proto_to_midi_file, note_sequence_to_pretty_midi

import magenta.music as mm
import tensorflow as tf


AttributeError: `RNNCell` is not available with Keras 3.

In [ ]:
# انتخاب کانفیگ مدل مناسب احساس کاربر
# برای مثال: attention_rnn برای آهنگ غمگین و پیچیده

from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle

bundle = sequence_generator_bundle.read_bundle_file('/content/attention_rnn.mag')
generator_map = melody_rnn_sequence_generator.get_generator_map()
generator = generator_map['attention_rnn'](checkpoint=None, bundle=bundle)
generator.initialize()

# ساخت آهنگ جدید
seed = mm.Melody([60])
input_sequence = seed.to_sequence(qpm=60)

generator_options = mm.generator_pb2.GeneratorOptions()
generator_options.generate_sections.add(
    start_time=0,
    end_time=30  # تولید 30 ثانیه موسیقی
)

sequence = generator.generate(input_sequence, generator_options)

# ذخیره خروجی
mm.sequence_proto_to_midi_file(sequence, '/content/generated_song.mid')


ModuleNotFoundError: No module named 'magenta'

In [ ]:
!sudo apt install fluidsynth
!pip install midi2audio music21


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin
  libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5
  libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom9 libxcb-icccm4 libxcb-image0 libxcb-keysyms1
  libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1
  libxkbcommon-x11-0 qsynth qt5-gtk-platformtheme qttranslations5-l10n
  timgm6mb-soundfont
Suggested packages:
  fluid-soundfont-gs qt5-image-formats-plugins qtwayland5 jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libevdev2 libfluidsynth3 libgudev-1.0-0
  libinput-bin libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom9 libx

In [ ]:
# emotion_music_generator.py

import random
from music21 import stream, note, tempo, key
from midi2audio import FluidSynth
import os

def generate_emotion_music(emotion_label='happy', length=16, save_path='emotion_music_output.mid'):
    """
    تولید موسیقی ساده با استفاده از music21 بر اساس برچسب احساس
    خروجی به صورت فایل MIDI ذخیره می‌شود
    """
    emotion_map = {
        'happy': {'scale': 'C', 'mode': 'major', 'bpm': 120},
        'sad': {'scale': 'A', 'mode': 'minor', 'bpm': 60},
        'angry': {'scale': 'E', 'mode': 'minor', 'bpm': 140},
        'calm': {'scale': 'D', 'mode': 'major', 'bpm': 70},
        'fear': {'scale': 'B', 'mode': 'minor', 'bpm': 100},
        'love': {'scale': 'G', 'mode': 'major', 'bpm': 90},
        'surprise': {'scale': 'F#', 'mode': 'major', 'bpm': 130},
    }

    config = emotion_map.get(emotion_label.lower(), emotion_map['happy'])

    s = stream.Stream()
    s.append(tempo.MetronomeMark(number=config['bpm']))
    s.append(key.Key(config['scale'], config['mode']))

    scale_notes = {
        'major': ['C', 'D', 'E', 'F', 'G', 'A', 'B'],
        'minor': ['A', 'B', 'C', 'D', 'E', 'F', 'G']
    }[config['mode']]

    for _ in range(length):
        pitch = random.choice(scale_notes)
        full_pitch = pitch + str(random.choice([4, 5]))
        n = note.Note(full_pitch)
        n.quarterLength = random.choice([0.5, 1.0])
        s.append(n)

    s.write('midi', fp=save_path)
    return save_path

def convert_midi_to_wav(midi_path, wav_path):
    """
    تبدیل فایل MIDI به WAV با استفاده از midi2audio + FluidSynth
    نیاز به نصب soundfont در مسیر /usr/share/sounds/sf2/FluidR3_GM.sf2
    """
    fs = FluidSynth(sound_font='/usr/share/sounds/sf2/FluidR3_GM.sf2')
    fs.midi_to_audio(midi_path, wav_path)
    return wav_path

if __name__ == '__main__':
    midi_file = generate_emotion_music('sad', length=32)
    print("Music generated:", midi_file)
    wav_file = convert_midi_to_wav(midi_file, 'emotion_music_output.wav')
    print("Converted to WAV:", wav_file)


Music generated: emotion_music_output.mid
Converted to WAV: emotion_music_output.wav


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.6 MB/s eta 0:00:00


In [ ]:
# gradio_app.py

import gradio as gr
import os
import tempfile

# تابع برای دریافت احساس و تولید موسیقی

def music_from_emotion(emotion):
    midi_path = tempfile.NamedTemporaryFile(delete=False, suffix=".mid").name
    wav_path = tempfile.NamedTemporaryFile(delete=False, suffix=".wav").name

    generate_emotion_music(emotion_label=emotion, save_path=midi_path)
    convert_midi_to_wav(midi_path, wav_path)

    return wav_path

# رابط گرافیکی Gradio
iface = gr.Interface(
    fn=music_from_emotion,
    inputs=gr.Dropdown(label="احساس را انتخاب کنید", choices=["happy", "sad", "angry", "calm", "fear", "love", "surprise"], value="happy"),
    outputs=gr.Audio(label="موسیقی تولید شده"),
    title="🎵 تولید موسیقی بر اساس احساسات",
    description="احساس خود را انتخاب کنید تا یک موسیقی مناسب همان حال و هوا ساخته شود."
)

if __name__ == "__main__":
    iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e4e0baa16c2cf9f95c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# UI

In [ ]:
!pip install gradio transformers torchaudio librosa opencv-python pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.0 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from transformers import pipeline
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from pydub import AudioSegment
import os

# --------------------------

# لود مدل‌های احساسات فارسی و انگلیسی
fa_model = pipeline("text-classification", model="HooshvareLab/bert-fa-base-uncased-sentiment-snappfood")
# fa_model = pipeline("text-classification", model="HooshvareLab/bert-fa-base-uncased-emotion")

en_model = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions")

def detect_lang(text):
    # تشخیص ساده زبان (فارسی یا انگلیسی)
    return 'fa' if any('\u0600' <= c <= '\u06FF' for c in text) else 'en'


# مدل احساس از صوت
audio_model = pipeline("audio-classification", model="superb/wav2vec2-base-superb-er")

# مدل احساس از چهره
face_model = load_model('emotion_model.h5')
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# --------------------------
# تابع تشخیص احساس چهره
def get_image_emotion(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    emotions = {}
    for (x, y, w, h) in faces:
        roi = gray[y:y+h, x:x+w]
        roi = cv2.resize(roi, (48, 48)) / 255.0
        roi = roi.reshape(1, 48, 48, 1)
        preds = face_model.predict(roi)[0]
        emotions = {emotion_labels[i]: float(preds[i]) for i in range(len(preds))}
    return emotions

# --------------------------
# ترکیب احساسات
from collections import defaultdict

def fuse_emotions(*emotion_dicts):
    total = defaultdict(float)
    for d in emotion_dicts:
        for emotion, score in d.items():
            total[emotion] += score
    return dict(sorted(total.items(), key=lambda x: x[1], reverse=True))

# --------------------------
# هسته پردازش کل سیستم
def analyze_emotion(text, audio_file, image_file):
    # تحلیل متن
    text_em = {i['label']: i['score'] for i in fa_model(text)} if text else {}

    # تحلیل صوت
    audio_path = audio_file
    audio = AudioSegment.from_file(audio_file)
    audio = audio.set_frame_rate(16000).set_channels(1)
    audio.export("temp.wav", format="wav")
    audio_em = {i['label']: i['score'] for i in audio_model("temp.wav")} if audio_file else {}

    # تحلیل تصویر
    img_em = {}
    if image_file:
        img_np = cv2.imdecode(np.frombuffer(image_file.read(), np.uint8), cv2.IMREAD_COLOR)
        img_em = get_image_emotion(img_np)

    # ترکیب احساسات
    fused = fuse_emotions(text_em, audio_em, img_em)
    top_emotion = list(fused.keys())[0] if fused else "unknown"

    # تعیین نوع موسیقی پیشنهادی
    music_suggestion = {
        'happy': "🎵 آهنگ شاد یا الکترونیک",
        'sad': "🎹 آهنگ پیانو آرام",
        'angry': "🥁 آهنگ پرانرژی یا راک",
        'fear': "🎼 موزیک امبینت آرام",
        'neutral': "🎶 لو فای یا کلاسیک ملایم"
    }.get(top_emotion.lower(), "🎧 موزیک عمومی")

    return top_emotion, fused, music_suggestion

# --------------------------
# رابط Gradio
gr.Interface(
    fn=analyze_emotion,
    inputs=[
        gr.Textbox(label="📝 متن دلخواه (فارسی)"),
        gr.Audio(label="🎤 ویس خود را آپلود کنید", type="filepath"),
        gr.File(label="🖼️ عکس چهره (JPG/PNG)")
    ],
    outputs=[
        gr.Textbox(label="🔮 احساس غالب شما"),
        gr.JSON(label="📊 جزئیات کامل احساسات"),
        gr.Textbox(label="🎼 پیشنهاد موسیقی")
    ],
    title="دستیار احساسی موسیقی هوشمند 🎧",
    description="با وارد کردن متن، صدا و عکس، احساست رو تشخیص می‌دیم و موزیک مناسب پیشنهاد یا تولید می‌کنیم!"
).launch()


OSError: HooshvareLab/bert-fa-base-uncased-emotion is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

##### Text UI

In [ ]:
from transformers import pipeline
import gradio as gr

# لود مدل‌های احساسات فارسی و انگلیسی
fa_model = pipeline("text-classification", model="HooshvareLab/bert-fa-base-uncased-sentiment-snappfood")
en_model = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions")

def detect_lang(text):
    # تشخیص ساده زبان (فارسی یا انگلیسی)
    return 'fa' if any('\u0600' <= c <= '\u06FF' for c in text) else 'en'

def analyze_text_emotion(text):
    lang = detect_lang(text)
    if lang == 'fa':
        result = fa_model(text)
    else:
        result = en_model(text)
    return result

# رابط Gradio فقط برای تست متن
gr.Interface(
    fn=analyze_text_emotion,
    inputs=gr.Textbox(label="📝 جمله‌تان را وارد کنید (فارسی یا انگلیسی)"),
    outputs="json",
    title="تحلیل احساسات متن (چندزبانه)",
    description="بررسی احساس جمله به زبان فارسی یا انگلیسی"
).launch()


# Train

##### DataSets

In [ ]:
# generate_dataset.py

import os
import random
from music21 import stream, note, tempo, key
import json
from tqdm import tqdm

# احساسات و توصیف متنی آن‌ها به همراه تنظیمات موسیقایی
emotion_map = {
    'happy': {
        'text': ['شاد', 'خوشحال', 'انرژیک'],
        'scale': 'C', 'mode': 'major', 'bpm': 120
    },
    'sad': {
        'text': ['غمگین', 'دلگیر', 'احساساتی'],
        'scale': 'A', 'mode': 'minor', 'bpm': 60
    },
    'angry': {
        'text': ['عصبانی', 'خشمگین'],
        'scale': 'E', 'mode': 'minor', 'bpm': 140
    },
    'calm': {
        'text': ['آرام', 'ریلکس'],
        'scale': 'D', 'mode': 'major', 'bpm': 70
    },
    'love': {
        'text': ['عاشقانه', 'احساس عشق'],
        'scale': 'G', 'mode': 'major', 'bpm': 90
    },
    'fear': {
        'text': ['ترس', 'نگرانی'],
        'scale': 'B', 'mode': 'minor', 'bpm': 100
    }
}

output_dir = "dataset"
os.makedirs(output_dir + "/midis", exist_ok=True)
os.makedirs(output_dir + "/json", exist_ok=True)

scale_notes = {
    'major': ['C', 'D', 'E', 'F', 'G', 'A', 'B'],
    'minor': ['A', 'B', 'C', 'D', 'E', 'F', 'G']
}

def create_sample(emotion_label, index):
    config = emotion_map[emotion_label]
    desc = random.choice(config['text'])

    s = stream.Stream()
    s.append(tempo.MetronomeMark(number=config['bpm']))
    s.append(key.Key(config['scale'], config['mode']))

    for _ in range(random.randint(16, 32)):
        pitch = random.choice(scale_notes[config['mode']])
        full_pitch = pitch + str(random.choice([4, 5]))
        n = note.Note(full_pitch)
        n.quarterLength = random.choice([0.5, 1.0])
        s.append(n)

    midi_path = f"{output_dir}/midis/{emotion_label}_{index}.mid"
    s.write('midi', fp=midi_path)

    sample = {
        "text": desc,
        "emotion": emotion_label,
        "midi_path": midi_path
    }
    json_path = f"{output_dir}/json/{emotion_label}_{index}.json"
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(sample, f, ensure_ascii=False, indent=2)

    return sample

if __name__ == "__main__":
    n_per_class = 50
    all_data = []
    for emotion in tqdm(emotion_map):
        for i in range(n_per_class):
            sample = create_sample(emotion, i)
            all_data.append(sample)

    with open(f"{output_dir}/dataset_index.json", "w", encoding="utf-8") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=2)

    print("✅ دیتاست ساخته شد!")

100%|██████████| 6/6 [00:18<00:00,  3.12s/it]

✅ دیتاست ساخته شد!


In [ ]:
# midi_tokenizer.py

from music21 import converter, note, chord
import os
import json

def midi_to_tokens(midi_path):
    """تبدیل فایل MIDI به دنباله‌ای از توکن‌ها"""
    midi = converter.parse(midi_path)
    tokens = []

    for el in midi.flat.notes:
        if isinstance(el, note.Note):
            tokens.append(f"NOTE_{el.pitch.nameWithOctave}")
            tokens.append(f"DUR_{el.quarterLength}")
        elif isinstance(el, chord.Chord):
            tokens.append(f"CHORD_{'.'.join(n.nameWithOctave for n in el.notes)}")
            tokens.append(f"DUR_{el.quarterLength}")

    return tokens

def tokenize_all(dataset_index_path, output_path):
    with open(dataset_index_path, "r", encoding="utf-8") as f:
        samples = json.load(f)

    tokenized_data = []
    for sample in samples:
        tokens = midi_to_tokens(sample["midi_path"])
        tokenized_data.append({
            "text": sample["text"],
            "emotion": sample["emotion"],
            "tokens": tokens
        })

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(tokenized_data, f, ensure_ascii=False, indent=2)

    print(f"✅ تمام فایل‌ها توکنیزه شدند و در '{output_path}' ذخیره شدند")

if __name__ == "__main__":
    tokenize_all("dataset/dataset_index.json", "dataset/tokenized_dataset.json")


/usr/local/lib/python3.11/dist-packages/music21/stream/base.py:3689: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


✅ تمام فایل‌ها توکنیزه شدند و در 'dataset/tokenized_dataset.json' ذخیره شدند


In [ ]:
# train_model.py

import json
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from torch.optim import AdamW
from tqdm import tqdm
import random

# تعریف دیتاست سفارشی
class TextToMusicDataset(Dataset):
    def __init__(self, path, tokenizer, max_len=128):
        with open(path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
        self.tokenizer = tokenizer
        self.max_len = max_len

        # ساخت واژگان توکن‌های موسیقی
        all_tokens = set()
        for item in self.data:
            all_tokens.update(item['tokens'])
        self.music_vocab = {tok: i+1 for i, tok in enumerate(sorted(all_tokens))}  # +1 برای reserved 0
        self.inv_music_vocab = {v: k for k, v in self.music_vocab.items()}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_text = item['text']
        input_ids = self.tokenizer(input_text, padding='max_length', truncation=True, max_length=self.max_len, return_tensors='pt')['input_ids'].squeeze(0)

        label_tokens = item['tokens'][:512]  # محدودیت طول برای ساده‌سازی
        label_ids = torch.tensor([self.music_vocab[tok] for tok in label_tokens] + [0]*(512 - len(label_tokens)))

        return input_ids, label_ids

# تعریف مدل ترنسفورمر ساده
class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size, emb_dim=256, nhead=4, nlayers=4):
        super().__init__()
        self.text_embedding = nn.Embedding(30522, emb_dim)  # اندازه BERT tokenizer
        self.music_embedding = nn.Embedding(vocab_size + 1, emb_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=emb_dim, nhead=nhead), num_layers=nlayers
        )
        self.fc_out = nn.Linear(emb_dim, vocab_size + 1)

    def forward(self, text_input, music_input):
        text_emb = self.text_embedding(text_input)  # (batch, seq1, emb)
        music_emb = self.music_embedding(music_input)  # (batch, seq2, emb)

        # concatenate embeddings instead of adding
        x = torch.cat((text_emb, music_emb), dim=1)
        x = x.permute(1, 0, 2)  # Transformer expects (seq, batch, emb)
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # back to (batch, seq, emb)

        # slice only the music part for prediction
        music_output = x[:, text_emb.size(1):, :]
        logits = self.fc_out(music_output)
        return logits

if __name__ == '__main__':
    from transformers import BertTokenizer

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    dataset = TextToMusicDataset("dataset/tokenized_dataset.json", tokenizer)
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SimpleTransformer(vocab_size=len(dataset.music_vocab)).to(device)
    optimizer = AdamW(model.parameters(), lr=5e-4)
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(5):
        model.train()
        total_loss = 0
        for text_input, label_ids in tqdm(dataloader):
            text_input = text_input.to(device)
            music_input = torch.zeros_like(label_ids).to(device)  # zero vector input
            label_ids = label_ids.to(device)

            optimizer.zero_grad()
            outputs = model(text_input, music_input)
            loss = loss_fn(outputs.view(-1, outputs.shape[-1]), label_ids.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"✅ Epoch {epoch+1} - Loss: {total_loss:.4f}")

    torch.save(model.state_dict(), "text_to_music_model.pth")
    with open("music_vocab.json", "w") as f:
        json.dump(dataset.music_vocab, f)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
100%|██████████| 38/38 [04:10<00:00,  6.58s/it]


✅ Epoch 1 - Loss: 23.6575


 21%|██        | 8/38 [00:53<03:20,  6.67s/it]


KeyboardInterrupt: 

In [ ]:
!apt-get install -y fluidsynth
!wget https://github.com/urish/cinelerra-soundfont/raw/master/FluidR3_GM.sf2 -P /usr/share/sounds/sf2/


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin
  libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5
  libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom9 libxcb-icccm4 libxcb-image0 libxcb-keysyms1
  libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1
  libxkbcommon-x11-0 qsynth qt5-gtk-platformtheme qttranslations5-l10n
  timgm6mb-soundfont
Suggested packages:
  fluid-soundfont-gs qt5-image-formats-plugins qtwayland5 jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libevdev2 libfluidsynth3 libgudev-1.0-0
  libinput-bin libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom9 libx

In [ ]:
# generate_music.py

import torch
import json
from transformers import BertTokenizer
from music21 import stream, note, midi, instrument
from pydub import AudioSegment
import os

# تعریف مدل ترنسفورمر ساده
class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size, emb_dim=256, nhead=4, nlayers=4):
        super().__init__()
        self.text_embedding = nn.Embedding(30522, emb_dim)  # اندازه BERT tokenizer
        self.music_embedding = nn.Embedding(vocab_size + 1, emb_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=emb_dim, nhead=nhead), num_layers=nlayers
        )
        self.fc_out = nn.Linear(emb_dim, vocab_size + 1)

    def forward(self, text_input, music_input):
        text_emb = self.text_embedding(text_input)  # (batch, seq1, emb)
        music_emb = self.music_embedding(music_input)  # (batch, seq2, emb)

        # concatenate embeddings instead of adding
        x = torch.cat((text_emb, music_emb), dim=1)
        x = x.permute(1, 0, 2)  # Transformer expects (seq, batch, emb)
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # back to (batch, seq, emb)

        # slice only the music part for prediction
        music_output = x[:, text_emb.size(1):, :]
        logits = self.fc_out(music_output)
        return logits


# تعریف دیتاست سفارشی
class TextToMusicDataset(Dataset):
    def __init__(self, path, tokenizer, max_len=128):
        with open(path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
        self.tokenizer = tokenizer
        self.max_len = max_len

        # ساخت واژگان توکن‌های موسیقی
        all_tokens = set()
        for item in self.data:
            all_tokens.update(item['tokens'])
        self.music_vocab = {tok: i+1 for i, tok in enumerate(sorted(all_tokens))}  # +1 برای reserved 0
        self.inv_music_vocab = {v: k for k, v in self.music_vocab.items()}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_text = item['text']
        input_ids = self.tokenizer(input_text, padding='max_length', truncation=True, max_length=self.max_len, return_tensors='pt')['input_ids'].squeeze(0)

        label_tokens = item['tokens'][:512]  # محدودیت طول برای ساده‌سازی
        label_ids = torch.tensor([self.music_vocab[tok] for tok in label_tokens] + [0]*(512 - len(label_tokens)))

        return input_ids, label_ids

with open("music_vocab.json", "w") as f:
        json.dump(dataset.music_vocab, f)
# بارگذاری واژگان موسیقی
with open("music_vocab.json", "r") as f:
    music_vocab = json.load(f)
inv_music_vocab = {v: k for k, v in music_vocab.items()}
vocab_size = len(music_vocab)

# بارگذاری مدل آموزش‌دیده
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleTransformer(vocab_size=vocab_size).to(device)
model.load_state_dict(torch.load("text_to_music_model.pth", map_location=device))
model.eval()

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def generate_tokens(text, max_len=512):
    input_ids = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=128)['input_ids'].to(device)
    music_input = torch.zeros((1, max_len), dtype=torch.long).to(device)
    with torch.no_grad():
        output = model(input_ids, music_input)
        predicted = torch.argmax(output, dim=-1).squeeze(0)
        tokens = [inv_music_vocab[str(tok.item())] for tok in predicted if tok.item() != 0]
    return tokens

def tokens_to_midi(tokens, out_file="output.mid"):
    s = stream.Stream()
    s.append(instrument.Piano())
    s.append(instrument.Guitar())
    s.append(instrument.Vibraphone())

    current_dur = 1.0
    for tok in tokens:
        if tok.startswith("NOTE_"):
            pitch = tok.replace("NOTE_", "")
            n = note.Note(pitch)
            n.quarterLength = current_dur
            s.append(n)
        elif tok.startswith("DUR_"):
            current_dur = float(tok.replace("DUR_", ""))

    if s.duration.quarterLength < 60:
        last_note = note.Rest()
        last_note.quarterLength = 60 - s.duration.quarterLength
        s.append(last_note)

    mf = midi.translate.streamToMidiFile(s)
    mf.open(out_file, 'wb')
    mf.write()
    mf.close()
    print(f"✅ MIDI ذخیره شد: {out_file}")

    os.system(f"fluidsynth -ni /usr/share/sounds/sf2/FluidR3_GM.sf2 {out_file} -F temp.wav -r 44100")
    AudioSegment.from_wav("temp.wav").export("output.mp3", format="mp3")
    print("🎵 MP3 نهایی ساخته شد: output.mp3")

if __name__ == '__main__':
    text_prompt = input("📝 متن احساسی رو وارد کن: ")
    tokens = generate_tokens(text_prompt)
    tokens_to_midi(tokens, "generated_music.mid")


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


📝 متن احساسی رو وارد کن: i feel happy
✅ MIDI ذخیره شد: generated_music.mid
🎵 MP3 نهایی ساخته شد: output.mp3


# 2

In [ ]:
# 📦 Phase 1: Data Preprocessing for All Modalities (Text, Audio, Image)

# ✅ Install required libraries
!pip install datasets librosa torchaudio torchvision opendatasets pandas kaggle --quiet

# ✅ Imports
import os
import librosa
import numpy as np
import pandas as pd
import torch
import torchaudio
import torchvision.transforms as T
from PIL import Image
from datasets import load_dataset

In [ ]:
# 📁 Create folders
os.makedirs("data/text_emotion", exist_ok=True)
os.makedirs("data/audio_emotion", exist_ok=True)
os.makedirs("data/image_emotion", exist_ok=True)

# ------------------------
# 🟢 TEXT EMOTION (GoEmotions)
# ------------------------
goemo = load_dataset("go_emotions", split="train")
goemo_df = pd.DataFrame(goemo)
goemo_df.to_csv("data/text_emotion/goemotions.csv", index=False)

# Simple preprocessing function
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
goemo_df["labels_vec"] = mlb.fit_transform(goemo_df["labels"]).tolist()

def preprocess_text(text):
    return text.lower().strip()

goemo_df["clean_text"] = goemo_df["text"].apply(preprocess_text)

# Save preprocessed version
goemo_df[["clean_text", "labels_vec"]].to_json("data/text_emotion/processed_text.json", orient="records")

# ------------------------
# 🟢 AUDIO EMOTION (RAVDESS via Zenodo)
# ------------------------
# 🔹 You must upload RAVDESS manually to data/audio_emotion or use opendatasets for Kaggle

AUDIO_DIR = "data/audio_emotion/ravdess/"
os.makedirs(AUDIO_DIR, exist_ok=True)

def extract_audio_features(file_path, sr=16000):
    y, sr = librosa.load(file_path, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mel = librosa.feature.melspectrogram(y=y, sr=sr)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    return {
        "mfcc": mfcc.T[:300],
        "mel": mel.T[:300],
        "chroma": chroma.T[:300]
    }

# Example for one file (you can loop through dataset later):
# feats = extract_audio_features("data/audio_emotion/ravdess/03-01-01-01-01-01-01.wav")

# ------------------------
# 🟢 IMAGE EMOTION (FER2013 via Kaggle)
# ------------------------
# 🔹 You must upload FER2013 CSV to data/image_emotion (fer2013.csv)

FER_CSV = "data/image_emotion/fer2013.csv"
if os.path.exists(FER_CSV):
    fer_df = pd.read_csv(FER_CSV)
    os.makedirs("data/image_emotion/images", exist_ok=True)

    def save_fer_image(pixels, usage, idx):
        arr = np.array([int(p) for p in pixels.split()], dtype=np.uint8).reshape(48, 48)
        img = Image.fromarray(arr)
        path = f"data/image_emotion/images/{usage}_{idx}.png"
        img.save(path)
        return path

    fer_df["img_path"] = fer_df.apply(lambda row: save_fer_image(row["pixels"], row["Usage"], row.name), axis=1)
    fer_df[["emotion", "Usage", "img_path"]].to_csv("data/image_emotion/processed_fer.csv", index=False)
else:
    print("❌ Please upload FER2013 csv file to data/image_emotion")

# Image transform for later use
image_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.5], std=[0.5])
])

# ------------------------
# ✅ Summary
print("✅ Text, Audio, and Image preprocessing scripts ready.")
print("➡️ Next: Build and train emotion recognition models for each modality.")


❌ Please upload FER2013 csv file to data/image_emotion
✅ Text, Audio, and Image preprocessing scripts ready.
➡️ Next: Build and train emotion recognition models for each modality.


In [ ]:
# 📦 Phase 2: Build Emotion Recognition Models (Text, Audio, Image)

# ✅ Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import librosa
from PIL import Image
import json

# ✅ Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------
# 🟢 TEXT EMOTION MODEL
# ------------------------
class TextDataset(Dataset):
    def __init__(self, json_path):
        with open(json_path, "r") as f:
            self.data = json.load(f)

    def __len__(self): return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]['clean_text']
        y = torch.tensor(self.data[idx]['labels_vec'], dtype=torch.float32)
        return x, y

from transformers import AutoTokenizer
text_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

class TextEmotionModel(nn.Module):
    def __init__(self, hidden_dim=128, output_dim=28):
        super().__init__()
        self.embedding = nn.Embedding(30522, 128)  # BERT's vocab size
        self.lstm = nn.LSTM(128, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_ids):
        embedded = self.embedding(x_ids)
        _, (h_n, _) = self.lstm(embedded)
        return torch.sigmoid(self.fc(h_n[-1]))

# ------------------------
# 🟢 AUDIO EMOTION MODEL
# ------------------------
class AudioDataset(Dataset):
    def __init__(self, folder):
        self.files = [f for f in os.listdir(folder) if f.endswith(".wav")]
        self.folder = folder

    def __len__(self): return len(self.files)

    def __getitem__(self, idx):
        path = os.path.join(self.folder, self.files[idx])
        y, sr = librosa.load(path, sr=16000)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        x = torch.tensor(mfcc.T[:300], dtype=torch.float32)
        label = int(self.files[idx].split("-")[2]) - 1
        y = F.one_hot(torch.tensor(label), num_classes=8).float()
        return x, y

class AudioEmotionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(13, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(128, 8)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv(x).squeeze(-1)
        return torch.softmax(self.fc(x), dim=-1)

# ------------------------
# 🟢 IMAGE EMOTION MODEL
# ------------------------
class ImageDataset(Dataset):
    def __init__(self, csv_file, transform):
        df = pd.read_csv(csv_file)
        self.paths = df["img_path"].tolist()
        self.labels = df["emotion"].tolist()
        self.transform = transform

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("L")
        img = self.transform(img)
        label = int(self.labels[idx])
        return img, F.one_hot(torch.tensor(label), num_classes=7).float()

class ImageEmotionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(32*56*56, 7)

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        return torch.softmax(self.fc(x), dim=-1)

# ------------------------
# ✅ Summary
print("✅ Emotion models for Text, Audio, and Image defined. Ready for training.")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

✅ Emotion models for Text, Audio, and Image defined. Ready for training.


In [ ]:
# 📦 Phase 2: Build Emotion Recognition Models (Text, Audio, Image)

# ✅ Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import librosa
from PIL import Image
import json
import os
from transformers import AutoTokenizer
import torchvision.transforms as T

# ✅ Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------
# 🟢 TEXT EMOTION MODEL
# ------------------------
class TextDataset(Dataset):
    def __init__(self, json_path, tokenizer):
        with open(json_path, "r") as f:
            self.data = json.load(f)
        self.tokenizer = tokenizer

    def __len__(self): return len(self.data)

    def __getitem__(self, idx):
        encoded = self.tokenizer(self.data[idx]['clean_text'], padding="max_length", truncation=True, max_length=64, return_tensors="pt")
        input_ids = encoded['input_ids'].squeeze(0)
        y = torch.tensor(self.data[idx]['labels_vec'], dtype=torch.float32)
        return input_ids, y

text_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

class TextEmotionModel(nn.Module):
    def __init__(self, hidden_dim=128, output_dim=28):
        super().__init__()
        self.embedding = nn.Embedding(30522, 128)  # BERT's vocab size
        self.lstm = nn.LSTM(128, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_ids):
        embedded = self.embedding(x_ids)
        _, (h_n, _) = self.lstm(embedded)
        return torch.sigmoid(self.fc(h_n[-1]))

# 🔁 Train Function

def train_model(model, dataloader, optimizer, criterion, epochs=3):
    model.train()
    model.to(device)
    for epoch in range(epochs):
        total_loss = 0
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1} | Loss: {total_loss / len(dataloader):.4f}")

# Train Text Model
text_ds = TextDataset("data/text_emotion/processed_text.json", text_tokenizer)
text_dl = DataLoader(text_ds, batch_size=32, shuffle=True)
text_model = TextEmotionModel()
text_optimizer = torch.optim.Adam(text_model.parameters(), lr=1e-3)
text_criterion = nn.BCELoss()
train_model(text_model, text_dl, text_optimizer, text_criterion, epochs=3)

# ------------------------
# 🟢 AUDIO EMOTION MODEL
# ------------------------
class AudioDataset(Dataset):
    def __init__(self, folder):
        self.files = [f for f in os.listdir(folder) if f.endswith(".wav")]
        self.folder = folder

    def __len__(self): return len(self.files)

    def __getitem__(self, idx):
        path = os.path.join(self.folder, self.files[idx])
        y, sr = librosa.load(path, sr=16000)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        x = torch.tensor(mfcc.T[:300], dtype=torch.float32)
        label = int(self.files[idx].split("-")[2]) - 1
        y = F.one_hot(torch.tensor(label), num_classes=8).float()
        return x, y

class AudioEmotionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(13, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(128, 8)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv(x).squeeze(-1)
        return torch.softmax(self.fc(x), dim=-1)

# Train Audio Model (Make sure RAVDESS is available)
audio_ds = AudioDataset("data/audio_emotion/ravdess")
audio_dl = DataLoader(audio_ds, batch_size=16, shuffle=True)
audio_model = AudioEmotionModel()
audio_optimizer = torch.optim.Adam(audio_model.parameters(), lr=1e-3)
audio_criterion = nn.CrossEntropyLoss()
train_model(audio_model, audio_dl, audio_optimizer, audio_criterion, epochs=3)

# ------------------------
# 🟢 IMAGE EMOTION MODEL
# ------------------------
class ImageDataset(Dataset):
    def __init__(self, csv_file, transform):
        df = pd.read_csv(csv_file)
        self.paths = df["img_path"].tolist()
        self.labels = df["emotion"].tolist()
        self.transform = transform

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("L")
        img = self.transform(img)
        label = int(self.labels[idx])
        return img, F.one_hot(torch.tensor(label), num_classes=7).float()

class ImageEmotionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(32*56*56, 7)

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        return torch.softmax(self.fc(x), dim=-1)

# Train Image Model
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.5], std=[0.5])
])

image_ds = ImageDataset("data/image_emotion/processed_fer.csv", transform)
image_dl = DataLoader(image_ds, batch_size=32, shuffle=True)
image_model = ImageEmotionModel()
image_optimizer = torch.optim.Adam(image_model.parameters(), lr=1e-3)
image_criterion = nn.CrossEntropyLoss()
train_model(image_model, image_dl, image_optimizer, image_criterion, epochs=3)

# ------------------------
# ✅ Summary
print("✅ Emotion models for Text, Audio, and Image trained successfully.")


Epoch 1 | Loss: 0.1534
Epoch 2 | Loss: 0.1500
Epoch 3 | Loss: 0.1500


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
# 📦 Phase 2: Build Emotion Recognition Models (Text, Audio, Image) + Fusion + Music Generator + Inference Pipeline

# ✅ Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import librosa
from PIL import Image
import json
import os
from transformers import AutoTokenizer
import torchvision.transforms as T

# ✅ Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------
# 🟢 TEXT EMOTION MODEL
# ------------------------
class TextDataset(Dataset):
    def __init__(self, json_path, tokenizer):
        with open(json_path, "r") as f:
            self.data = json.load(f)
        self.tokenizer = tokenizer

    def __len__(self): return len(self.data)

    def __getitem__(self, idx):
        encoded = self.tokenizer(self.data[idx]['clean_text'], padding="max_length", truncation=True, max_length=64, return_tensors="pt")
        input_ids = encoded['input_ids'].squeeze(0)
        y = torch.tensor(self.data[idx]['labels_vec'], dtype=torch.float32)
        return input_ids, y

text_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

class TextEmotionModel(nn.Module):
    def __init__(self, hidden_dim=128, output_dim=28):
        super().__init__()
        self.embedding = nn.Embedding(30522, 128)  # BERT's vocab size
        self.lstm = nn.LSTM(128, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_ids):
        embedded = self.embedding(x_ids)
        _, (h_n, _) = self.lstm(embedded)
        return torch.sigmoid(self.fc(h_n[-1]))

# 🔁 Train Function

def train_model(model, dataloader, optimizer, criterion, epochs=3):
    model.train()
    model.to(device)
    for epoch in range(epochs):
        total_loss = 0
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1} | Loss: {total_loss / len(dataloader):.4f}")

# Train Text Model
text_ds = TextDataset("data/text_emotion/processed_text.json", text_tokenizer)
text_dl = DataLoader(text_ds, batch_size=32, shuffle=True)
text_model = TextEmotionModel()
text_optimizer = torch.optim.Adam(text_model.parameters(), lr=1e-3)
text_criterion = nn.BCELoss()
train_model(text_model, text_dl, text_optimizer, text_criterion, epochs=3)

# ------------------------
# 🟢 AUDIO EMOTION MODEL
# ------------------------
class AudioDataset(Dataset):
    def __init__(self, folder):
        self.files = [f for f in os.listdir(folder) if f.endswith(".wav")]
        self.folder = folder

    def __len__(self): return len(self.files)

    def __getitem__(self, idx):
        path = os.path.join(self.folder, self.files[idx])
        y, sr = librosa.load(path, sr=16000)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        x = torch.tensor(mfcc.T[:300], dtype=torch.float32)
        label = int(self.files[idx].split("-")[2]) - 1
        y = F.one_hot(torch.tensor(label), num_classes=8).float()
        return x, y

class AudioEmotionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(13, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(128, 8)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv(x).squeeze(-1)
        return torch.softmax(self.fc(x), dim=-1)

# Train Audio Model
audio_ds = AudioDataset("data/audio_emotion/ravdess")
audio_dl = DataLoader(audio_ds, batch_size=16, shuffle=True)
audio_model = AudioEmotionModel()
audio_optimizer = torch.optim.Adam(audio_model.parameters(), lr=1e-3)
audio_criterion = nn.CrossEntropyLoss()
train_model(audio_model, audio_dl, audio_optimizer, audio_criterion, epochs=3)

# ------------------------
# 🟢 IMAGE EMOTION MODEL
# ------------------------
class ImageDataset(Dataset):
    def __init__(self, csv_file, transform):
        df = pd.read_csv(csv_file)
        self.paths = df["img_path"].tolist()
        self.labels = df["emotion"].tolist()
        self.transform = transform

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("L")
        img = self.transform(img)
        label = int(self.labels[idx])
        return img, F.one_hot(torch.tensor(label), num_classes=7).float()

class ImageEmotionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(32*56*56, 7)

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        return torch.softmax(self.fc(x), dim=-1)

# Train Image Model
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.5], std=[0.5])
])

image_ds = ImageDataset("data/image_emotion/processed_fer.csv", transform)
image_dl = DataLoader(image_ds, batch_size=32, shuffle=True)
image_model = ImageEmotionModel()
image_optimizer = torch.optim.Adam(image_model.parameters(), lr=1e-3)
image_criterion = nn.CrossEntropyLoss()
train_model(image_model, image_dl, image_optimizer, image_criterion, epochs=3)

# ------------------------
# 🔁 FUSION: Combine Modalities
# ------------------------
class FusionEmotionModel(nn.Module):
    def __init__(self, text_dim=28, audio_dim=8, image_dim=7):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(text_dim + audio_dim + image_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 10)  # Assuming 10 final emotion/music categories
        )

    def forward(self, text_feat, audio_feat, image_feat):
        x = torch.cat([text_feat, audio_feat, image_feat], dim=1)
        return torch.softmax(self.fc(x), dim=-1)

# ------------------------
# 🎵 MUSIC GENERATOR (Placeholder)
# ------------------------
class SimpleMusicGenerator(nn.Module):
    def __init__(self, input_dim=10):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.Tanh()  # For now output is dummy waveform
        )

    def forward(self, emotion_vector):
        return self.fc(emotion_vector)

# ------------------------
# 🚀 INFERENCE PIPELINE
# ------------------------
def run_inference(text, audio_path, image_path):
    text_ids = text_tokenizer(text, padding="max_length", truncation=True, max_length=64, return_tensors="pt")["input_ids"].to(device)
    text_feat = text_model(text_ids).detach()

    y, sr = librosa.load(audio_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    audio_input = torch.tensor(mfcc.T[:300], dtype=torch.float32).unsqueeze(0).to(device)
    audio_feat = audio_model(audio_input).detach()

    image = Image.open(image_path).convert("L")
    image_tensor = transform(image).unsqueeze(0).to(device)
    image_feat = image_model(image_tensor).detach()

    fusion_model = FusionEmotionModel()
    fusion_model.to(device)
    emotion_vec = fusion_model(text_feat, audio_feat, image_feat)

    music_gen = SimpleMusicGenerator()
    music_gen.to(device)
    music_out = music_gen(emotion_vec)
    return music_out

# ✅ Summary
print("✅ Emotion models for Text, Audio, Image + Fusion + Music Generator + Inference pipeline setup done.")


In [ ]:
import torch
import librosa
from transformers import AutoTokenizer
from PIL import Image
import torchvision.transforms as T

# Assume models and preprocessing functions are already loaded from previous code

# -------------------
# 1. Load Pre-trained Models
# -------------------
text_model = TextEmotionModel()
text_model.load_state_dict(torch.load('text_emotion_model.pth'))
text_model.to(device).eval()

audio_model = AudioEmotionModel()
audio_model.load_state_dict(torch.load('audio_emotion_model.pth'))
audio_model.to(device).eval()

image_model = ImageEmotionModel()
image_model.load_state_dict(torch.load('image_emotion_model.pth'))
image_model.to(device).eval()

fusion_model = FusionEmotionModel()
fusion_model.load_state_dict(torch.load('fusion_emotion_model.pth'))
fusion_model.to(device).eval()

music_generator = SimpleMusicGenerator()
music_generator.load_state_dict(torch.load('music_generator_model.pth'))
music_generator.to(device).eval()

# -------------------
# 2. Preprocessing Functions
# -------------------

# Text Preprocessing
text_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_text(text):
    encoded = text_tokenizer(text, padding="max_length", truncation=True, max_length=64, return_tensors="pt")
    input_ids = encoded['input_ids'].squeeze(0)
    return input_ids

# Audio Preprocessing
def preprocess_audio(audio_path):
    y, sr = librosa.load(audio_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_tensor = torch.tensor(mfcc.T[:300], dtype=torch.float32).unsqueeze(0)
    return mfcc_tensor

# Image Preprocessing
transform = T.Compose([T.Resize((224, 224)), T.ToTensor(), T.Normalize(mean=[0.5], std=[0.5])])

def preprocess_image(image_path):
    img = Image.open(image_path).convert("L")
    img = transform(img).unsqueeze(0)
    return img

# -------------------
# 3. Inference Functions
# -------------------

def predict_text_emotion(text):
    input_ids = preprocess_text(text).to(device)
    with torch.no_grad():
        output = text_model(input_ids)
    return output

def predict_audio_emotion(audio_path):
    audio_tensor = preprocess_audio(audio_path).to(device)
    with torch.no_grad():
        output = audio_model(audio_tensor)
    return output

def predict_image_emotion(image_path):
    image_tensor = preprocess_image(image_path).to(device)
    with torch.no_grad():
        output = image_model(image_tensor)
    return output

def generate_music_from_emotion(emotion_vector):
    with torch.no_grad():
        music_output = music_generator(emotion_vector)
    return music_output

# -------------------
# 4. Fusion + Final Prediction
# -------------------

def predict_and_generate_music(text, audio_path, image_path):
    # Step 1: Predict emotions from each modality
    text_emotion = predict_text_emotion(text)
    audio_emotion = predict_audio_emotion(audio_path)
    image_emotion = predict_image_emotion(image_path)

    # Step 2: Fuse the emotions (combine features)
    fused_emotion = fusion_model(text_emotion, audio_emotion, image_emotion)

    # Step 3: Generate music based on the fused emotion vector
    music = generate_music_from_emotion(fused_emotion)

    return music

# -------------------
# 5. Test Example
# -------------------

text_input = "I am feeling excited and happy!"
audio_input = "path_to_audio.wav"
image_input = "path_to_image.jpg"

music_output = predict_and_generate_music(text_input, audio_input, image_input)

# You can save or visualize the generated music output
print("Generated Music Output:", music_output)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import librosa
from transformers import AutoTokenizer
from PIL import Image
import torchvision.transforms as T

# -------------------
# 1. مدل‌های تشخیص احساس از متن، صوت و تصویر
# -------------------

# مدل تشخیص احساس از متن
class TextEmotionModel(nn.Module):
    def __init__(self):
        super(TextEmotionModel, self).__init__()
        self.bert = AutoTokenizer.from_pretrained('distilbert-base-uncased')  # استفاده از DistilBERT برای مدل متن
        self.fc = nn.Linear(768, 10)  # خروجی 10 کلاس احساس

    def forward(self, input_ids):
        # عملکرد پیش‌بینی احساس از متن
        output = self.bert(input_ids)
        logits = self.fc(output.last_hidden_state.mean(dim=1))  # میانگین گیری از همه کلمات
        return logits

# مدل تشخیص احساس از صوت
class AudioEmotionModel(nn.Module):
    def __init__(self):
        super(AudioEmotionModel, self).__init__()
        self.lstm = nn.LSTM(input_size=13, hidden_size=64, num_layers=2, batch_first=True)
        self.fc = nn.Linear(64, 10)

    def forward(self, audio_features):
        lstm_out, _ = self.lstm(audio_features)
        output = self.fc(lstm_out[:, -1, :])  # استفاده از خروجی آخر LSTM
        return output

# مدل تشخیص احساس از تصویر
class ImageEmotionModel(nn.Module):
    def __init__(self):
        super(ImageEmotionModel, self).__init__()
        self.resnet = torchvision.models.resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 10)

    def forward(self, image_tensor):
        return self.resnet(image_tensor)

# مدل ادغام احساسات
class FusionEmotionModel(nn.Module):
    def __init__(self):
        super(FusionEmotionModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(30, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, text_emotion, audio_emotion, image_emotion):
        fusion = torch.cat((text_emotion, audio_emotion, image_emotion), dim=1)  # ادغام ویژگی‌ها
        return self.fc(fusion)

# مدل تولید موسیقی
class SimpleMusicGenerator(nn.Module):
    def __init__(self, input_dim=10):
        super(SimpleMusicGenerator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.Tanh()  # تبدیل به داده‌های موسیقی
        )

    def forward(self, emotion_vector):
        return self.fc(emotion_vector)

# -------------------
# 2. پیش‌پردازش داده‌ها
# -------------------

# پیش‌پردازش متن
text_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_text(text):
    encoded = text_tokenizer(text, padding="max_length", truncation=True, max_length=64, return_tensors="pt")
    input_ids = encoded['input_ids'].squeeze(0)
    return input_ids

# پیش‌پردازش صوت
def preprocess_audio(audio_path):
    y, sr = librosa.load(audio_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_tensor = torch.tensor(mfcc.T[:300], dtype=torch.float32).unsqueeze(0)
    return mfcc_tensor

# پیش‌پردازش تصویر
transform = T.Compose([T.Resize((224, 224)), T.ToTensor(), T.Normalize(mean=[0.5], std=[0.5])])

def preprocess_image(image_path):
    img = Image.open(image_path).convert("L")
    img = transform(img).unsqueeze(0)
    return img

# -------------------
# 3. بارگذاری مدل‌ها
# -------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

text_model = TextEmotionModel().to(device)
audio_model = AudioEmotionModel().to(device)
image_model = ImageEmotionModel().to(device)
fusion_model = FusionEmotionModel().to(device)
music_generator = SimpleMusicGenerator().to(device)

# بارگذاری مدل‌ها (وزن‌ها)
text_model.load_state_dict(torch.load('text_emotion_model.pth'))
audio_model.load_state_dict(torch.load('audio_emotion_model.pth'))
image_model.load_state_dict(torch.load('image_emotion_model.pth'))
fusion_model.load_state_dict(torch.load('fusion_emotion_model.pth'))
music_generator.load_state_dict(torch.load('music_generator_model.pth'))

# -------------------
# 4. پیش‌بینی و تولید موسیقی
# -------------------

def predict_text_emotion(text):
    input_ids = preprocess_text(text).to(device)
    with torch.no_grad():
        output = text_model(input_ids)
    return output

def predict_audio_emotion(audio_path):
    audio_tensor = preprocess_audio(audio_path).to(device)
    with torch.no_grad():
        output = audio_model(audio_tensor)
    return output

def predict_image_emotion(image_path):
    image_tensor = preprocess_image(image_path).to(device)
    with torch.no_grad():
        output = image_model(image_tensor)
    return output

def generate_music_from_emotion(emotion_vector):
    with torch.no_grad():
        music_output = music_generator(emotion_vector)
    return music_output

def predict_and_generate_music(text, audio_path, image_path):
    # پیش‌بینی احساسات از هر مدالیتی
    text_emotion = predict_text_emotion(text)
    audio_emotion = predict_audio_emotion(audio_path)
    image_emotion = predict_image_emotion(image_path)

    # ادغام احساسات
    fused_emotion = fusion_model(text_emotion, audio_emotion, image_emotion)

    # تولید موسیقی از احساسات
    music = generate_music_from_emotion(fused_emotion)

    return music

# -------------------
# 5. مثال استفاده
# -------------------

text_input = "I am feeling excited and happy!"
audio_input = "path_to_audio.wav"
image_input = "path_to_image.jpg"

music_output = predict_and_generate_music(text_input, audio_input, image_input)

print("Generated Music Output:", music_output)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FileNotFoundError: [Errno 2] No such file or directory: 'text_emotion_model.pth'